<a href="https://colab.research.google.com/github/SebastianAblerKratkey/YFinance-Download-test/blob/main/backtest_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install numpy_financial
#!pip install mplfinance
#!pip install adjustText
#!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import numpy_financial as npf
import pandas as pd
from pandas_datareader.data import DataReader as dr
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FuncFormatter
from matplotlib import gridspec
import mplfinance as mpf
import seaborn as sns
import scipy.stats as stats
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.stats import norm
import os
import datetime
import statsmodels.api as sm
import base64
from io import StringIO, BytesIO

# this module is utilized to prevent the annotations in the plot from overlapping
from adjustText import adjust_text

# Get Yahoo Finance Data
import yfinance as yf

# Library for Website creation
import streamlit as st
st.set_option('deprecation.showPyplotGlobalUse', False)

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

def convert_date_index(df):
    # Convert the index to datetime
    df.index = pd.to_datetime(df.index)
    # Extract the month and year from the datetime
    df.index = df.index.strftime("%b %Y")
    return df

def create_performance_index(price_df):
    returns = price_df.pct_change()
    growth = returns+1
    growth = growth.fillna(1) # set starting value for index
    index = growth.cumprod()
    index = index - 1 # deduct starting value to get the percentage change
    return index

def visualize_performance(prices, list_of_names):
    benchmarking_data = create_performance_index(prices)

    color_list = ['deepskyblue', 'steelblue', 'mediumslateblue', 'cornflowerblue', 'lightsteelblue',
                    'mediumslateblue', 'lightblue']

    benchmarking_data_filtered = benchmarking_data.filter(list_of_names)
    if len(list_of_names) > 0:
        benchmarking_data_filtered.plot(figsize=(15, 10), color=color_list)
    else:
        plt.figure(figsize=(15, 10))


    plt.fill_between(benchmarking_data.index, benchmarking_data.max(axis=1), benchmarking_data.min(axis=1),
                        color='grey', alpha=0.17, label="Range of all assets")

    # Calculate the number of days to add
    num_days = (benchmarking_data_filtered.index.max() - benchmarking_data_filtered.index.min()).days
    days_to_add1 = num_days / 120
    days_to_add2 = num_days / 12

    # Plot scatter points at the end of each line
    for col in benchmarking_data_filtered.columns:
        plt.scatter(benchmarking_data_filtered.index[-1], benchmarking_data_filtered[col].iloc[-1], color=color_list[list_of_names.index(col)], zorder=5)
        #text lablel is offset by a number of days to the right
        plt.text(benchmarking_data_filtered.index[-1] + pd.Timedelta(days=days_to_add1), benchmarking_data_filtered[col].iloc[-1], str(round(benchmarking_data_filtered[col].iloc[-1]*100, 2))+"%",color=color_list[list_of_names.index(col)], size=12, verticalalignment='bottom')

    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    plt.gca().xaxis.set_major_locator(MaxNLocator())
    plt.gca().set_xlim(left=benchmarking_data.head(1).index.max())



    plt.xlim(right=benchmarking_data.index.max() + pd.Timedelta(days=days_to_add2))  # Extend x-axis limit by number of days
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))  # Format dates to show month and year
    plt.grid('on', ls="--")
    plt.ylabel(f"Performance (indexed: {benchmarking_data.head(1).index.max().strftime('%d.%m.%Y')} = 0%)", fontsize=12)
    plt.legend(fontsize=12)

    # Rotate x-axis labels to be horizontal
    plt.xticks(rotation=0, ha='center')

    # Remove x-axis label
    plt.gca().set_xlabel('')

    plt.show()

def visualize_summary(summary):
    fontsize=8
    plt.rc('font', size=fontsize)
    fig, (ax1, ax2) = plt.subplots(1, 2, clip_on=False)
    ax1.grid('on', ls="--")
    ax1.set_axisbelow(True)
    ax1.yaxis.grid(False)
    ax2.grid('on', ls="--")
    ax2.set_axisbelow(True)
    ax2.yaxis.grid(False)
    ax3 =  ax2.twiny()
    ax4 = ax1.twiny()
    ax1.xaxis.set_major_locator(MaxNLocator(nbins="auto"))
    ax2.xaxis.set_major_locator(MaxNLocator(nbins="auto"))
    ax3.xaxis.set_major_locator(MaxNLocator(nbins="auto"))
    ax4.xaxis.set_major_locator(MaxNLocator(prune='upper', nbins="auto"))
    x_dim = max(max(summary["mean return"]), max(summary['standard deviation'])) * 1.1
    height_of_fig = len(summary)*0.1
    ax1.set_position([0, 0, 0.35, height_of_fig])
    ax2.set_position([0.35, 0, 0.35, height_of_fig])
    ax1.set_xlim(left=0, right=x_dim)
    ax2.set_xlim(left=0, right=x_dim)
    ax3.set_xlim(left=0, right=x_dim)
    ax4.set_xlim(left=-x_dim, right=0)
    ax1.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax2.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax3.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax4.xaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))
    ax1.invert_xaxis()
    ax2.tick_params(left = False, bottom=False)
    ax2.set_yticklabels([])
    ax2.set_xticklabels([])
    summary_sorted = summary.copy()
    summary_sorted["r/std"] = summary["mean return"] / summary['standard deviation']
    summary_sorted.sort_values("r/std", inplace=True)
    bar_width = 0.6  # Set a fixed width for the horizontal bars
    for index, row in summary_sorted.iterrows():
        ax1.barh(index, row['standard deviation'], height=bar_width, color="steelblue")
        ax2.barh(index,  row['mean return'], height=bar_width, color="deepskyblue")
        if row['mean return'] < 0:
            if abs(row['mean return']) > abs(row['standard deviation']):
                ax1.barh(index, abs(row['mean return']), height=bar_width, color="deepskyblue")
                ax1.barh(index, row['standard deviation'], height=bar_width, color="steelblue")
            if abs(row['mean return']) <= abs(row['standard deviation']):
                ax1.barh(index, row['standard deviation'], height=bar_width, color="steelblue")
                ax1.barh(index, abs(row['mean return']), height=bar_width, color="deepskyblue")
    ax1_patch = mpatches.Patch(color='deepskyblue', label='Mean return')
    ax1.legend(handles=[ax1_patch], fontsize=fontsize, frameon=False, loc='center', ncol=2, bbox_to_anchor=(1, 1+0.8/len(summary)))
    ax2_patch = mpatches.Patch(color='steelblue', label='Volatility')
    ax2.legend(handles=[ax2_patch], fontsize=fontsize, frameon=False, loc='center', ncol=2, bbox_to_anchor=(0, -0.8/len(summary)))
    plt.show()


def visualize_correlation(corr):
    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["deepskyblue", "mediumslateblue", "slategrey"])
    mask = np.triu(corr, k=1)
    plt.figure(figsize=(12, 7))
    sns.heatmap(corr, annot=True, cmap=cmap, mask=mask, linewidths=5,
                annot_kws={'color':'white'})
    plt.show()

def portfolio_std(weights):
    portfolio_std = np.sum(weights * np.sum(weights * annualized_cov_returns, axis=1)) ** 0.5
    return portfolio_std

def portfolio_return(weights, returns):
    portfolio_return = np.sum(weights * returns)
    return portfolio_return

def negative_portfolio_SR(weights, rf, returns):
    return_p = portfolio_return(weights, returns)
    std_p = portfolio_std(weights)
    negative_sharpe_ratio = -1*(return_p - rf) / std_p
    return negative_sharpe_ratio

def negative_portfolio_utility(weights, returns):
    return_p = portfolio_return(weights, returns)
    std_p = portfolio_std(weights)
    negative_portfolio_utility = -1*(return_p - 0.5*A*std_p**2)
    return negative_portfolio_utility

def create_KPI_report(name, weights, rf, returns):
    KPIs = pd.DataFrame(index=[name])
    KPIs["portfolio return"] = portfolio_return(weights, returns)
    KPIs["protfolio std"] = portfolio_std(weights)
    KPIs["sharpe ratio"] = (KPIs["portfolio return"]- rf) / KPIs["protfolio std"]
    KPIs["utility"] = KPIs["portfolio return"] - 0.5*A*KPIs["protfolio std"]**2
    return KPIs

def create_portfolio_visual(name, summary, KPIs):
    plt.figure(figsize=(8, 8))
    plt.pie(summary["weight"], wedgeprops=dict(width=0.45),
            colors=['deepskyblue', 'steelblue', 'lightblue', 'lightsteelblue', 'cornflowerblue',
                    'mediumslateblue','thistle', 'dodgerblue', 'slategrey'],
            autopct='%.2f%%',pctdistance=0.8, startangle=90,labels=summary.index)
    plt.annotate(name, xy=(0,0), fontsize=30, va="center", ha="center")
    plt.annotate("E(r): {}%".format(float((KPIs["portfolio return"]*100).round(decimals=2))),
                 xy=(-0.07,-0.18), fontsize=10, va="center", ha="right")
    plt.annotate("Vola: {}%".format(float((KPIs["protfolio std"]*100).round(decimals=2))),
                 xy=(+0.07,-0.18), fontsize=10, va="center", ha="left")
    plt.show()

def create_mvf_cal_visual():
    #plot minimum varriance frontier and CAL
    color1 = 'cornflowerblue'
    color2 = 'darkmagenta'

    plt.figure(figsize=(15, 10))

    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter('{:,.2%}'.format))
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter('{:,.2%}'.format))
    plt.gca().set_xlim(left=0)
    plt.gca().set_xlim(right=max(max(summary["standard deviation"]),float(KPIs_ocp["protfolio std"]))*1.05)

    plt.scatter(summary["standard deviation"], summary["mean return"], color=color1)

    # capital allocation line

    # between std = 0 and std = std_orp_l
    std_cal_1 = np.arange(0, float(KPIs_orp_l["protfolio std"]), step)
    return_cal_1 = rf_l + float(KPIs_orp_l["sharpe ratio"])*std_cal_1
    plt.plot(std_cal_1 ,return_cal_1, color=color1, label='Capital allocation line')

    # between std_orp_l and std_orp_b -> follows minimum varriance frontier
    mvf_plot_slice = mvf_plot_data[(mvf_plot_data["return"] >= float(KPIs_orp_l["portfolio return"])) &
                               (mvf_plot_data["return"] <= float(KPIs_orp_b["portfolio return"]))]
    std_cal_2 = mvf_plot_slice["std"]
    return_cal_2 = mvf_plot_slice["return"]
    plt.plot(std_cal_2,return_cal_2,color=color1)

    # after std_orp_b
    endpoint_cal = plt.gca().get_xlim()[1]
    std_cal_3 = np.arange(float(KPIs_orp_b["protfolio std"]), endpoint_cal, step)
    return_cal_3 = rf_b + float(KPIs_orp_b["sharpe ratio"])*std_cal_3
    plt.plot(std_cal_3 ,return_cal_3, color=color1)

    # minimum varriance frontier
    plt.plot(mvf_plot_data["std"], mvf_plot_data["return"], color=color1, linestyle='--',
         label='Minimum varriance frontier')

    plt.scatter(KPIs_mvp["protfolio std"], KPIs_mvp["portfolio return"], color=color2)
    plt.scatter(KPIs_orp["protfolio std"], KPIs_orp["portfolio return"], color=color2)
    plt.scatter(KPIs_ocp["protfolio std"], KPIs_ocp["portfolio return"], color=color2)

    plt.legend(fontsize=12)
    plt.xlabel("Volatility", fontsize=12)
    plt.ylabel("Mean return", fontsize=12)
    plt.grid('on', ls="--")

    # labeling
    x_offset = plt.gca().get_xlim()[1]*0.01
    for i in summary_p.index:
        plt.annotate(i,(summary_p["protfolio std"][i], summary_p["portfolio return"][i]),
                 (summary_p["protfolio std"][i]-x_offset, summary_p["portfolio return"][i]),
                 color=color2, fontsize=12, ha='right')

    labels = []
    for i in summary.index:
        labels.append(plt.text(summary["standard deviation"][i], summary["mean return"][i], i, size=8))
    adjust_text(labels)

    plt.show()

def currency_formatter_alt_EUR_decimal_seperator(x, currency="EUR"):
         if currency == 'EUR':
            return f'{currency} {x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", ".")
         elif currency == 'USD':
            return f'{currency} {x:,.2f}'

def currency_formatter(x, currency="EUR"):
    return f'{currency} {x:,.2f}'

def currency_formatter_signs(x, currency="EUR"):
         if currency == 'EUR':
            return f'€ {x:,.2f}'
         elif currency == 'USD':
            return f'$ {x:,.2f}'

def visualize_simulaiton(sim_avg, deposits, currency='EUR'):
    """
    Plots the average simulated performance over time.

    Parameters:
    sim_avg (DataFrame): A DataFrame containing the average simulated performance.
    currency (str): The currency in which to display the performance data.

    Returns:
    None
    """
    def currency_formatter(x, pos):
         if currency == 'EUR':
            return f'€ {x:,.2f}'
         elif currency == 'USD':
            return f'$ {x:,.2f}'

    plt.figure(figsize=(15, 5))

    # Set the y-axis formatter
    plt.gca().yaxis.set_major_formatter(FuncFormatter(currency_formatter))

    # Set the tick locations and labels
    plt.xticks(sim_avg.index)
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter('{:.0f}'.format))

    # Plot the bars
    if sim_avg.iloc[-1] > deposits[-1]:
        plt.bar(sim_avg.index, sim_avg, color='deepskyblue', label="Capital", align='center')
        plt.bar(sim_avg.index, deposits, color='steelblue', label="Money invested", align='center')
    if sim_avg.iloc[-1] < deposits[-1]:
        plt.bar(sim_avg.index, deposits, color='steelblue', label="Money invested", align='center')
        plt.bar(sim_avg.index, sim_avg, color='deepskyblue', label="Capital", align='center')

    # Set the x-axis limits based on the minimum and maximum values in the index
    plt.gca().set_xlim(left=sim_avg.index.min()-0.8)
    plt.gca().set_xlim(right=sim_avg.index.max()+0.8)

    # Rotate xticks if needed
    if len(sim_avg.index) > 22:
        plt.xticks(rotation=45)
    if len(sim_avg.index) > 60:
        plt.xticks(rotation=90)
    if len(sim_avg.index) > 75:
        plt.gca().tick_params(axis='x', labelsize=8)

    plt.gca().set_axisbelow(True)
    plt.grid('on', ls="--")
    plt.gca().xaxis.grid(False)
    plt.legend(fontsize=12)

    plt.show()

def generate_excel_download_link(df):
    towrite = BytesIO()
    df.to_excel(towrite, index=False, header=True)
    towrite.seek(0)
    b64 = base64.b64encode(towrite.read()).decode()
    href = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="template.xlsx">Excel template'
    return st.markdown(href, unsafe_allow_html=True)

def maximum_drawdowns(price_df):
    """
    Calculate the maximum drawdowns of a dataframe of asset prices.

    Parameters:
    price_df (pd.DataFrame): A pandas DataFrame containing asset prices.
    (date index must be sorted ascending)

    Returns:
    pd.Series: Series of asset names and corresponding maximum drawdowns.
    """
    price_df_sorted = price_df.sort_index(ascending=True)
    max_price_df = price_df_sorted.rolling(window=len(price_df_sorted),min_periods=1).max()
    dd_price_df = price_df_sorted / max_price_df -1
    max_dd_series = dd_price_df.min()

    return max_dd_series

def get_monthly_closing_prices(price_df_daily):
    price_df_monthly = price_df_daily.loc[price_df_daily.groupby(price_df_daily.index.to_period('M')).apply(lambda x: x.index.max())]
    return price_df_monthly


def simulate_leveraged_daily_compounded_annual_return(daily_return,
                                                      daily_vola,
                                                      leverage,
                                                      reference_rate,
                                                      expense_ratio,
                                                      assumed_trading_days,
                                                      sim_runs):
    delta_t = 1/assumed_trading_days
    daily_leverage_cost = ((leverage-1)*reference_rate + expense_ratio)*delta_t

    # run monte carlo simmulation
    daily_return_sim = np.log(1 + leverage*(daily_return + daily_vola*np.random.normal(0, 1, size=(sim_runs, assumed_trading_days))) - daily_leverage_cost)

    daily_compounded_annual_returns = np.sum(daily_return_sim, axis=1)

    mean_daily_compounded_annual_return = daily_compounded_annual_returns.mean()
    std_daily_compounded_annual_return = daily_compounded_annual_returns.std()

    return mean_daily_compounded_annual_return, std_daily_compounded_annual_return

def create_leverage_sim_visual(results_df):
    # Create figure and axis objects
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot mean return on primary y-axis
    ax1.scatter(results_df['Leverage'], results_df['Mean_Return'], label='Simulated return', color='cornflowerblue')
    ax1.set_xlabel('Leverage')
    ax1.set_ylabel('Daily compounded annual return')
    ax1.yaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))

    plt.grid('on', ls="--")
    # Create secondary y-axis for standard deviation
    ax2 = ax1.twinx()
    ax2.scatter(results_df['Leverage'], results_df['Std_Return'], label='Simulated volatility', color='darkmagenta')
    ax2.set_ylabel('Volatility of annual returns')
    ax2.yaxis.set_major_formatter(plt.FuncFormatter('{:,.0%}'.format))

    # ask matplotlib for the plotted objects and their labels
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    plt.legend(lines + lines2, labels + labels2, loc=0)

    # Display the plot
    plt.show()

def create_binary_colormap_for_plt_charts(data_values, two_color_list):

    cmap = matplotlib.colors.ListedColormap(two_color_list)

    # scale data
    denominator = max(data_values) - min(data_values)
    scaled_data = [(datum-min(data_values))/denominator for datum in data_values]

    colors = []
    for decimal in scaled_data:
        colors.append(cmap(decimal))

    return colors

def create_colormap_for_plt_charts(data_values, color_list):

    cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", color_list)

    # scale data
    denominator = max(data_values) - min(data_values)
    scaled_data = [(datum-min(data_values))/denominator for datum in data_values]

    colors = []
    for decimal in scaled_data:
        colors.append(cmap(decimal))

    return colors

#Technical Analysis functions
def calculate_macd(data, price="Close", days_fast=12, days_slow=26, days_signal=9):
    short_ema = data[price].ewm(span=days_fast, adjust=False).mean()
    long_ema = data[price].ewm(span=days_slow, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=days_signal, adjust=False).mean()
    macd_hist = macd - signal
    return short_ema, long_ema, macd, signal, macd_hist

def pandas_rsi(df: pd.DataFrame, window_length: int = 14, output: str = None, price: str = 'Close'):
    """
    An implementation of Wells Wilder's RSI calculation as outlined in
    his 1978 book "New Concepts in Technical Trading Systems" which makes
    use of the α-1 Wilder Smoothing Method of calculating the average
    gains and losses across trading periods and the Pandas library.

    @author: https://github.com/alphazwest
    Args:
        df: pandas.DataFrame - a Pandas Dataframe object
        window_length: int - the period over which the RSI is calculated. Default is 14
        output: str or None - optional output path to save data as CSV
        price: str - the column name from which the RSI values are calcuated. Default is 'Close'

    Returns:
        DataFrame object with columns as such, where xxx denotes an inconsequential
        name of the provided first column:
            ['xxx', 'diff', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs', 'rsi']
    """
    # Calculate Price Differences using the column specified as price.
    df['diff1'] = df[price].diff(1)

    # Calculate Avg. Gains/Losses
    df['gain'] = df['diff1'].clip(lower=0).round(2)
    df['loss'] = df['diff1'].clip(upper=0).abs().round(2)

    # Get initial Averages
    df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]
    df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=window_length).mean()[:window_length+1]

    # Calculate Average Gains
    for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
        df['avg_gain'].iloc[i + window_length + 1] =\
            (df['avg_gain'].iloc[i + window_length] *
             (window_length - 1) +
             df['gain'].iloc[i + window_length + 1])\
            / window_length

    # Calculate Average Losses
    for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
        df['avg_loss'].iloc[i + window_length + 1] =\
            (df['avg_loss'].iloc[i + window_length] *
             (window_length - 1) +
             df['loss'].iloc[i + window_length + 1])\
            / window_length

    # Calculate RS Values
    df['rs'] = df['avg_gain'] / df['avg_loss']

    # Calculate RSI
    df['rsi'] = 100 - (100 / (1.0 + df['rs']))

    # Save if specified
    if output is not None:
        df.to_csv(output)

    return df

In [51]:
asset_name = "BTC-USD"
asset_data = yf.download(asset_name)

days_back_period = len(asset_data)

days_ema = 200

asset_data = asset_data.tail(days_back_period)

period_RSI=14
asset_data = pandas_rsi(df=asset_data, window_length=period_RSI, price="Close")
asset_data["macd_short_ema"], asset_data["macd_long_ema"], asset_data["macd"], asset_data["macd_signal"], asset_data["macd_hist"] = calculate_macd(asset_data, price="Close", days_fast=12, days_slow=26, days_signal=9)
asset_data["70_line"] = 70
asset_data["30_line"] = 30

# calculate addplot data
asset_data["ema"] = asset_data["Close"].ewm(span=days_ema, adjust=False).mean()

asset_data["daily_return"] = asset_data["Close"].pct_change()
asset_data = asset_data.dropna()



[*********************100%%**********************]  1 of 1 completed
<ipython-input-2-3b0261a9b9db>:463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff1'] = df[price].diff(1)
<ipython-input-2-3b0261a9b9db>:466: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gain'] = df['diff1'].clip(lower=0).round(2)
<ipython-input-2-3b0261a9b9db>:467: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [52]:
def get_cumulatieve_investment_values(returns, payments):
  sum_cum_values = pd.Series(0, index=payments.index, dtype=float)

  for i in payments[payments > 0].index:
    returns_i = returns.loc[i:]
    growth_i = 1 + returns_i
    growth_i[0] = payments[i]
    cum_value_i = growth_i.cumprod()

    sum_cum_values = sum_cum_values.add(cum_value_i, fill_value=0)

  return sum_cum_values


def apply_investment_signal(returns, starting_value, signal):
  date_index = signal.index
  values = pd.Series(starting_value, index=date_index, dtype=float)

  for i in range(1, len(date_index)):
    if signal.iloc[i] == 1:
      values.iloc[i] = values.iloc[i-1] * (1+returns.iloc[i])
    else:
      values.iloc[i] = values.iloc[i-1]

  return values

In [53]:
end_of_month_dates = get_monthly_closing_prices(asset_data).index


asset_data["monthly_payments"] = 0
asset_data.loc[asset_data.index.isin(end_of_month_dates), "monthly_payments"] = 100 / len(end_of_month_dates)
asset_data["start_100"] = 0
asset_data["start_100"][0] = 100


asset_data["benchmark_monthly_payments"] = get_cumulatieve_investment_values(asset_data["daily_return"], asset_data["monthly_payments"])
asset_data["benchmark_payment_t0"] = get_cumulatieve_investment_values(asset_data["daily_return"], asset_data["start_100"])

# MACD trigger and signal
asset_data["macd_trigger"] = np.where((asset_data['macd_hist'] > 0) & (asset_data['macd_hist'].shift(1) <= 0), 1, np.where((asset_data['macd_hist'] < 0) & (asset_data['macd_hist'].shift(1) >= 0), -1,0))
asset_data["macd_signal"] = asset_data["macd_trigger"].cumsum().shift(1)
# Normalize the signals to 0,1 format
if asset_data["macd_signal"].min(skipna=True) < 0:
  asset_data["macd_signal"] = asset_data["macd_signal"] + 1

asset_data["macd_backtest"] = apply_investment_signal(asset_data["daily_return"], 100, asset_data["macd_signal"])

# RSI trigger and signal
# Initialize the trigger column with zeros
asset_data["rsi_trigger"] = 0

# Identify the conditions for trigger changes
cross_70 = (asset_data['rsi'] <= 70) & (asset_data['rsi'].shift(1) > 70)
cross_30 = (asset_data['rsi'] >= 30) & (asset_data['rsi'].shift(1) < 30)

# Loop through the rows and update the trigger column
prev_trigger = 0  # Initialize previous trigger value
for index, row in asset_data.iterrows():
    if cross_70[index]:
        if prev_trigger != -1:  # Check if previous trigger was not -1
            asset_data.at[index, "rsi_trigger"] = -1
            prev_trigger = -1
    elif cross_30[index]:
        if prev_trigger != 1:  # Check if previous trigger was not +1
            asset_data.at[index, "rsi_trigger"] = 1
            prev_trigger = 1

asset_data["rsi_signal"] = asset_data["rsi_trigger"].cumsum().shift(1)
# Normalize the signals to 0,1 format
if asset_data["rsi_signal"].min(skipna=True) < 0:
  asset_data["rsi_signal"] = asset_data["rsi_signal"] + 1

asset_data["rsi_backtest"] = apply_investment_signal(asset_data["daily_return"], 100, asset_data["rsi_signal"])



asset_data

<ipython-input-53-d65560909d1a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data["start_100"][0] = 100


,Open,High,Low,Close,Adj Close,Volume,diff1,gain,loss,avg_gain,...,daily_return,monthly_payments,start_100,benchmark_monthly_payments,benchmark_payment_t0,macd_trigger,macd_backtest,rsi_trigger,rsi_signal,rsi_backtest
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-01,387.4270,391.3790,380.7800,383.6150,383.6150,26229400,-3.3290,0.0000,3.3300,4.4686,...,-0.0086,0.0000,100,0.0000,100.0000,0,100.0000,0,NaN,100.0000
2014-10-02,383.9880,385.4970,372.9460,375.0720,375.0720,21777700,-8.5430,0.0000,8.5400,4.1494,...,-0.0223,0.0000,0,0.0000,97.7730,0,100.0000,0,0.0000,100.0000
2014-10-03,375.1810,377.6950,357.8590,359.5120,359.5120,30901200,-15.5600,0.0000,15.5600,3.8530,...,-0.0415,0.0000,0,0.0000,93.7169,0,100.0000,0,0.0000,100.0000
2014-10-04,359.8920,364.4870,325.8860,328.8660,328.8660,47236500,-30.6460,0.0000,30.6500,3.5778,...,-0.0852,0.0000,0,0.0000,85.7281,0,100.0000,0,0.0000,100.0000
2014-10-05,328.9160,341.8010,289.2960,320.5100,320.5100,83308096,-8.3560,0.0000,8.3600,3.3222,...,-0.0254,0.0000,0,0.0000,83.5499,0,100.0000,0,0.0000,100.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-02,62431.6523,62458.6992,61657.2891,62029.8477,62029.8477,23888473685,-410.7852,0.0000,410.7900,934.8952,...,-0.0066,0.0000,0,4752.0287,16169.8185,0,33324.3039,0,0.0000,607.9316
2024-03-03,62031.5781,63230.2109,61435.0234,63167.3711,63167.3711,26253811450,1137.5234,1137.5200,0.0000,949.3684,...,0.0183,0.0000,0,4839.1729,16466.3459,0,33935.4157,0,0.0000,607.9316
2024-03-04,63137.0039,68537.0312,62386.5195,68330.4141,68330.4141,70670471105,5163.0430,5163.0400,0.0000,1250.3449,...,0.0817,0.0000,0,5234.7071,17812.2377,0,36709.1581,0,0.0000,607.9316


In [54]:
asset_data.to_excel("asset_data.xlsx", index=True)

In [31]:
# Initialize the trigger column with zeros
asset_data["rsi_trigger"] = 0

# Identify the conditions for trigger changes
cross_70 = (asset_data['rsi'] <= 70) & (asset_data['rsi'].shift(1) > 70)
cross_30 = (asset_data['rsi'] >= 30) & (asset_data['rsi'].shift(1) < 30)

# Loop through the rows and update the trigger column
prev_trigger = 0  # Initialize previous trigger value
for index, row in asset_data.iterrows():
    if cross_70[index]:
        if prev_trigger != -1:  # Check if previous trigger was not -1
            asset_data.at[index, "rsi_trigger"] = -1
            prev_trigger = -1
    elif cross_30[index]:
        if prev_trigger != 1:  # Check if previous trigger was not +1
            asset_data.at[index, "rsi_trigger"] = 1
            prev_trigger = 1

print(asset_data["rsi_trigger"].head(60))

Date
1981-01-05    0
1981-01-06   -1
1981-01-07    0
1981-01-08    0
1981-01-09    0
1981-01-12    0
1981-01-13    0
1981-01-14    0
1981-01-15    0
1981-01-16    0
1981-01-19    0
1981-01-20    0
1981-01-21    0
1981-01-22    0
1981-01-23    0
1981-01-26    0
1981-01-27    0
1981-01-28    0
1981-01-29    0
1981-01-30    0
1981-02-02    0
1981-02-03    0
1981-02-04    0
1981-02-05    0
1981-02-06    0
1981-02-09    0
1981-02-10    0
1981-02-11    0
1981-02-12    0
1981-02-13    0
1981-02-17    0
1981-02-18    1
1981-02-19    0
1981-02-20    0
1981-02-23    0
1981-02-24    0
1981-02-25    0
1981-02-26    0
1981-02-27    0
1981-03-02    0
1981-03-03    0
1981-03-04    0
1981-03-05    0
1981-03-06    0
1981-03-09    0
1981-03-10    0
1981-03-11    0
1981-03-12    0
1981-03-13    0
1981-03-16    0
1981-03-17    0
1981-03-18    0
1981-03-19    0
1981-03-20    0
1981-03-23    0
1981-03-24    0
1981-03-25    0
1981-03-26    0
1981-03-27    0
1981-03-30    0
Name: rsi_trigger, dtype: int64


In [ ]:
newvalue= (max'-min')/(max-min)*(value-max)+max'
normalized_values = (values - values.min(skipna=True)) / (values.max(skipna=True) - values.min(skipna=True))


In [ ]:
date_index = asset_data.index
returns = asset_data["daily_return"]
value = 100

# investment signal -1, 0, 1
asset_data["trigger"] = 0
asset_data["trigger"][4] = 1
asset_data["trigger"][20] = -1
asset_data["trigger"][25] = 1
asset_data["signal"] = asset_data["trigger"].cumsum().shift(1)
asset_data

<ipython-input-66-9501d89674e8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data["trigger"][4] = 1
<ipython-input-66-9501d89674e8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data["trigger"][20] = -1
<ipython-input-66-9501d89674e8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data["trigger"][25] = 1


,Open,High,Low,Close,Adj Close,Volume,diff1,gain,loss,avg_gain,...,70_line,30_line,ema,daily_return,monthly_payments,start_100,benchmark_monthly_payments,benchmark_payment_t0,trigger,signal
Date,,,,,,,,,,,,,,,,,,,,,
1981-01-05,0.1512,0.1512,0.1507,0.1507,0.1164,35728000,-0.0033,0.0000,0.0000,0.0036,...,70,30,0.1298,NaN,0.0000,100,0.0000,100.0000,0,NaN
1981-01-06,0.1445,0.1445,0.1440,0.1440,0.1113,45158400,-0.0067,0.0000,0.0100,0.0033,...,70,30,0.1299,-0.0444,0.0000,0,0.0000,95.5552,0,0.0000
1981-01-07,0.1384,0.1384,0.1378,0.1378,0.1065,55686400,-0.0061,0.0000,0.0100,0.0031,...,70,30,0.1300,-0.0426,0.0000,0,0.0000,91.4814,0,0.0000
1981-01-08,0.1356,0.1356,0.1350,0.1350,0.1044,39827200,-0.0028,0.0000,0.0000,0.0029,...,70,30,0.1301,-0.0202,0.0000,0,0.0000,89.6297,0,0.0000
1981-01-09,0.1423,0.1429,0.1423,0.1423,0.1100,21504000,0.0073,0.0100,0.0000,0.0034,...,70,30,0.1302,0.0537,0.0000,0,0.0000,94.4441,1,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-28,182.5100,183.1200,180.1300,181.4200,181.4200,48953900,-1.2100,0.0000,1.2100,0.5706,...,70,30,180.8112,-0.0066,0.0000,0,47320.8635,120408.8338,0,1.0000
2024-02-29,181.2700,182.5700,179.5300,180.7500,180.7500,136682600,-0.6700,0.0000,0.6700,0.5298,...,70,30,180.8106,-0.0037,0.1927,0,47146.2966,119964.1546,0,1.0000
2024-03-01,179.5500,180.5300,177.3800,179.6600,179.6600,73488000,-1.0900,0.0000,1.0900,0.4920,...,70,30,180.7992,-0.0060,0.0000,0,46861.9851,119240.7218,0,1.0000


In [ ]:
date_index = asset_data.index
returns = asset_data["daily_return"]
starting_value = 100
signal = asset_data["macd_signal"]
values = pd.Series(100, index=date_index, dtype=float)

for i in date_index[1:]:
  if signal[i] == 1:
    values[i] = values[i] * (1+returns[i])
  else:
    values[i] = values.iloc[values.index.get_loc(i)-1]


values

Date
1981-01-05   100.0000
1981-01-06    95.5552
1981-01-07    95.7367
1981-01-08    97.9758
1981-01-09    97.9758
               ...   
2024-02-28    98.0642
2024-02-29    98.0642
2024-03-01    98.0642
2024-03-04    98.0642
2024-03-05    98.0642
Length: 10883, dtype: float64

In [ ]:
def apply_investment_signal(returns, starting_value, signal):
  date_index = signal.index
  values = pd.Series(starting_value, index=date_index, dtype=float)

  for i in range(1, len(date_index)):
    if signal.iloc[i] == 1:
      values.iloc[i] = values.iloc[i-1] * (1+returns.iloc[i])
    else:
      values.iloc[i] = values.iloc[i-1]

  return values


Date
1981-01-05     100.0000
1981-01-06      95.5552
1981-01-07      91.4814
1981-01-08      89.6297
1981-01-09      89.6297
                ...    
2024-02-28   31474.6716
2024-02-29   31474.6716
2024-03-01   31474.6716
2024-03-04   31474.6716
2024-03-05   31474.6716
Length: 10883, dtype: float64

In [ ]:
asset_data["daily_return"] = asset_data["Close"].pct_change()

asset_data["payments_1"] = 0
asset_data["payments_1"][1] = 100
asset_data["payments_1"][4] = 200

In [ ]:
# backtesting

asset_data["daily_return"] = asset_data["Close"].pct_change()

asset_data["payments_1"] = 0
asset_data["payments_1"][1] = 100
asset_data["payments_1"][4] = 200


def get_cumulatieve_investment_values(returns, payments):
  sum_cum_values = pd.Series(0, index=payments.index, dtype=float)

  for i in payments[payments > 0].index:
    returns_i = returns.loc[i:]
    growth_i = 1 + returns_i
    growth_i[0] = payments[i]
    cum_value_i = growth_i.cumprod()

    sum_cum_values = sum_cum_values.add(cum_value_i, fill_value=0)

  return sum_cum_values



asset_data["test1"] = 1+ asset_data["daily_return"]
asset_data["test1"][1] = 100
asset_data["test1"] = asset_data["test1"].cumprod()

asset_data["test2"] = 1 + asset_data["daily_return"]
asset_data["test2"][4] = 200
asset_data["test2"].iloc[:4] = 1
asset_data["test2"] = asset_data["test2"].cumprod()

asset_data["test3"] = asset_data["test1"]+asset_data["test2"]

asset_data["test"] = get_cumulatieve_investment_values(asset_data["daily_return"], asset_data["payments_1"])
asset_data["test_"] = get_cumulatieve_investment_values(asset_data["daily_return"], asset_data["payments_1"])

asset_data

<ipython-input-103-40122982f117>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data["payments_1"][1] = 100
<ipython-input-103-40122982f117>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data["payments_1"][4] = 200
<ipython-input-103-40122982f117>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data["test1"][1] = 100
<ipython-input-103-40122982f117>:30: SettingWithCopyWarning: 
A value is trying to be set on a 

,Open,High,Low,Close,Adj Close,Volume,diff1,gain,loss,avg_gain,...,macd_hist,70_line,30_line,ema,daily_return,payments_1,test,test1,test2,test3
Date,,,,,,,,,,,,,,,,,,,,,
1981-01-05,0.1512,0.1512,0.1507,0.1507,0.1164,35728000,-0.0033,0.0000,0.0000,0.0036,...,0.0026,70,30,0.1298,NaN,0,0.0000,NaN,1.0000,NaN
1981-01-06,0.1445,0.1445,0.1440,0.1440,0.1113,45158400,-0.0067,0.0000,0.0100,0.0033,...,0.0015,70,30,0.1299,-0.0444,100,100.0000,100.0000,1.0000,101.0000
1981-01-07,0.1384,0.1384,0.1378,0.1378,0.1065,55686400,-0.0061,0.0000,0.0100,0.0031,...,0.0004,70,30,0.1300,-0.0426,0,95.7367,95.7367,1.0000,96.7367
1981-01-08,0.1356,0.1356,0.1350,0.1350,0.1044,39827200,-0.0028,0.0000,0.0000,0.0029,...,-0.0006,70,30,0.1301,-0.0202,0,93.7988,93.7988,1.0000,94.7988
1981-01-09,0.1423,0.1429,0.1423,0.1423,0.1100,21504000,0.0073,0.0100,0.0000,0.0034,...,-0.0008,70,30,0.1302,0.0537,200,298.8373,98.8373,200.0000,298.8373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-27,181.1000,183.9200,179.5600,182.6300,182.6300,54318900,1.4700,1.4700,0.0000,0.6144,...,-0.3445,70,30,180.8051,0.0081,0,383535.0640,126850.1829,256684.8810,383535.0640
2024-02-28,182.5100,183.1200,180.1300,181.4200,181.4200,48953900,-1.2100,0.0000,1.2100,0.5706,...,-0.3171,70,30,180.8112,-0.0066,0,380993.9700,126009.7429,254984.2272,380993.9700
2024-02-29,181.2700,182.5700,179.5300,180.7500,180.7500,136682600,-0.6700,0.0000,0.6700,0.5298,...,-0.3116,70,30,180.8106,-0.0037,0,379586.9297,125544.3791,254042.5506,379586.9297
